##### Context and Content

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target


##### Features

enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

gender: Gender of candidate

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

training_hours: training hours completed

target: 0 – Not looking for job change, 1 – Looking for a job change

Inspiration
Predict the probability of a candidate will work for the company
Interpret model(s) such a way that illustrate which features affect candidate decision

##### Goal of the project

Prepare the model to predict whether someone is willing to change the job.

##### Minor goals
- Investigate the data
- Understand the data - search for insights, clues
- Preprocess the data, so it can be used later on by a machine learning model

In [4]:
# import the pandas library

import pandas as pd

In [14]:
# read the file with training data

file = 'pandas_scikit_learn/data/aug_train.csv'
df = pd.read_csv(file)

FileNotFoundError: [Errno 2] No such file or directory: 'pandas_scikit_learn/data/aug_train.csv'

In [6]:
# let's see the first 5 rows of the dataset

df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


##### Examining the data

We need to get a grasp of how big the dataset is. Also, we need to know how many cases we have for both cases we want to predict:
- people who want to change their job
- people who want to stay at their current workplace

In [7]:
df.describe()

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


# Question: Did this output tell us anything? Conclusions?

In [6]:
df.shape

(19158, 14)

# Question: Is this enough? Can we expect to have our algorithm work properly? What else do we want to check?

In [7]:
df['target'] == 0

0        False
1         True
2         True
3        False
4         True
         ...  
19153    False
19154    False
19155     True
19156     True
19157     True
Name: target, Length: 19158, dtype: bool

In [8]:
df['target'] == 1

0         True
1        False
2        False
3         True
4        False
         ...  
19153     True
19154     True
19155    False
19156    False
19157    False
Name: target, Length: 19158, dtype: bool

In [8]:
df[df['target'] == 0].shape

(14381, 14)

In [9]:
df[df['target'] == 1].shape

(4777, 14)

# Question: What do we notice here? What does it tell us? Why it's important from the algorithm perspective?

### Investigation of columns
Let's investigate separate columns one by one.

##### Two ways of manipulating pandas dataframes:
- assign output of a function to a variable
- set *inplace* flag to True

In [24]:
# 1) sorted_df = df.set_index('enrollee_id').sort_index()
# 2) df.set_index('enrollee_id', inplace=True)

In [28]:
df = df.set_index('enrollee_id')

In [29]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


# Question: Look at the rest of the columns: please answer what dtypes do their represent?

### Let's investigate the unique values for columns containing categorical variables

In [30]:
df.columns

Index(['city', 'city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours', 'target'],
      dtype='object')

In [31]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [32]:
categorical_columns = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

### Verify what kind of unique values exist in each column:

In [35]:
for col in categorical_columns:
    unique_items = df[col].unique().tolist()
    print(f'col name: {col}, unique_items: {unique_items}')

col name: city, unique_items: ['city_103', 'city_40', 'city_21', 'city_115', 'city_162', 'city_176', 'city_160', 'city_46', 'city_61', 'city_114', 'city_13', 'city_159', 'city_102', 'city_67', 'city_100', 'city_16', 'city_71', 'city_104', 'city_64', 'city_101', 'city_83', 'city_105', 'city_73', 'city_75', 'city_41', 'city_11', 'city_93', 'city_90', 'city_36', 'city_20', 'city_57', 'city_152', 'city_19', 'city_65', 'city_74', 'city_173', 'city_136', 'city_98', 'city_97', 'city_50', 'city_138', 'city_82', 'city_157', 'city_89', 'city_150', 'city_70', 'city_175', 'city_94', 'city_28', 'city_59', 'city_165', 'city_145', 'city_142', 'city_26', 'city_12', 'city_37', 'city_43', 'city_116', 'city_23', 'city_99', 'city_149', 'city_10', 'city_45', 'city_80', 'city_128', 'city_158', 'city_123', 'city_7', 'city_72', 'city_106', 'city_143', 'city_78', 'city_109', 'city_24', 'city_134', 'city_48', 'city_144', 'city_91', 'city_146', 'city_133', 'city_126', 'city_118', 'city_9', 'city_167', 'city_27',

### See how many nans exist in each column:

In [46]:
from pandas import isnull
df_is_na_value = df.applymap(lambda x: isnull(x))

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
8949,False,False,False,False,False,False,False,False,True,True,False,False,False
29725,False,False,False,False,False,False,False,False,False,False,False,False,False
11561,False,False,True,False,False,False,False,False,True,True,False,False,False
33241,False,False,True,False,True,False,False,False,True,False,False,False,False
666,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7386,False,False,False,False,False,False,False,False,True,True,False,False,False
31398,False,False,False,False,False,False,False,False,True,True,False,False,False
24576,False,False,False,False,False,False,False,False,False,False,False,False,False


In [48]:
df_is_na_value = df.apply(lambda x: x.isna())

In [50]:
df_is_na_value.sum(axis=0)

city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [52]:
df_is_na_value.sum(axis=1)

enrollee_id
8949     2
29725    0
11561    3
33241    3
666      0
        ..
7386     2
31398    2
24576    0
5756     1
23834    4
Length: 19158, dtype: int64

# Question: What do we do with those nan values?

### A quick remark on speed issues in pandas
Let's assume we want to perform a very simple operation - we need to raise the values in the 'city_development_index' column to the second power. Let's try three approaches:
- loops
- apply() method
- builtin vectorized pandas method

##### Loop

In [56]:
%%timeit

for _, record in df.iterrows():
    record['city_development_index'] ** 2

417 ms ± 31.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### apply()

In [53]:
%%timeit

df['city_development_index'].apply(lambda x: x**2)

2.29 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### vectorized builtin method

In [54]:
%%timeit

df['city_development_index'] ** 2

40.1 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
